# 1. Формулировка задачи

| Характеристика | Описание |
| --- | --- |
| **Суть задачи** | Агент получает матрицу смежности взвешенного графа, начальную и конечную вершины. Ему нужно найти кратчайший путь. |
| **Регулировка сложности** | Параметр `difficulty` (от 1 до 10) маппится на гиперпараметры графа: количество вершин (например, `N = difficulty * 2 + 3`) и плотность ребер. |
| **Верификация (Verifier)** | Классический алгоритм Дейкстры рассчитывает эталонный кратчайший путь и его стоимость. Ответ агента парсится, суммируются веса ребер предложенного маршрута, и проверяется, совпадает ли эта сумма с минимальной, а также существует ли такой путь в принципе. |

# 2. Импорты

In [1]:
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1" # [NEW] Extra 30% context lengths!
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    pass # For Colab / Kaggle, we need extra instructions hidden below \/

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy, PIL; _numpy = f'numpy=={numpy.__version__}'; _pil = f'pillow=={PIL.__version__}'
    except: _numpy = "numpy"; _pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    _vllm, _triton = ('vllm==0.9.2', 'triton==3.2.0') if is_t4 else ('vllm==0.15.1', 'triton')
    !uv pip install -qqq --upgrade {_vllm} {_numpy} {_pil} torchvision bitsandbytes xformers unsloth
    !uv pip install -qqq {_triton}
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

In [3]:
%cd /content
!rm -rf HW-2_env
!git clone https://github.com/TebelevGt/HW-2_env.git
%cd HW-2_env/rl-shortest-path-agent

/content
Cloning into 'HW-2_env'...
remote: Enumerating objects: 319, done.
remote: Counting objects: 100% (319/319), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 319 (delta 145), reused 295 (delta 122), pack-reused 0 (from 0)
Receiving objects: 100% (319/319), 4.23 MiB | 7.75 MiB/s, done.
Resolving deltas: 100% (145/145), done.
/content/HW-2_env/rl-shortest-path-agent


In [4]:
from envs import PathEnv, PathVerifier, ShortestPathDataset, get_shortest_path_dataset
import os
import re

In [5]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch

# Увеличиваем контекст, так как reasoning-цепочки могут быть длинными
max_seq_length = 4096
lora_rank = 64

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-1.5B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.8, # Чуть снизил для запаса под генерацию (rollouts)
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = 128,     # Рекомендуется: alpha = rank * 2
    lora_dropout = 0.05,  # Добавляем регуляризацию
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True,    # Включаем Rank-Stabilized LoRA для лучшей сходимости
)

# ВАЖНО: Для Instruct моделей нужно убедиться, что токенизатор настроен корректно
# Unsloth обычно делает это сам, но для генерации (inference) в RL полезно явно задать:
tokenizer.padding_side = "left" # Для генерации обычно left, для обучения right (Unsloth сам меняет при SFT, но для RL важно проверить)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 02-21 23:28:48 [__init__.py:244] Automatically detected platform cuda.
ERROR 02-21 23:28:51 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-21 23:29:04 [vllm_utils.py:723] Unsloth: Patching vLLM v1 graph capture
INFO 02-21 23:29:04 [vllm_utils.py:752] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2026.2.1: Fast Qwen2 patching. Transformers: 4.56.2. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM l

`torch_dtype` is deprecated! Use `dtype` instead!


WARNING 02-21 23:29:25 [config.py:3371] Casting torch.bfloat16 to torch.float16.
INFO 02-21 23:29:25 [config.py:1472] Using max model len 4096
WARNING 02-21 23:29:25 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 02-21 23:29:26 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=4096.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'float16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_use_double_quant': True, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': ['lm_head', 'multi_modal_projector', 'merger', 'modality_projection', 'model.layers.0.self_attn', 'model.layers.1.self_attn', 'model.layers.2.mlp', 'model.layers.3.mlp', 'model.layers.4.mlp', 'model.layers.25.mlp', 'model.layers.26.mlp'], 'llm_int8_threshold': 6.0}
INFO 02-21 23:29:26 [llm_engine.py

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2026.2.1 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from trl import GRPOTrainer,GRPOConfig
from envs import *

training_args = GRPOConfig(
    use_vllm=True,  # use vllm for fast inference!
    learning_rate=5e-6,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="adamw_8bit",
    logging_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,  # Increase to 4 for smoother training
    num_generations=8,  # Decrease if out of memory
    max_prompt_length=256,
    max_completion_length=200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps=250,
    save_steps=250,
    max_grad_norm=0.1,
    report_to="none",  # Can use Weights & Biases
    output_dir="outputs",
)

trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        xmlcount_reward_func,        # Проверяет базовое наличие тегов (мягко)
        soft_format_reward_func,     # Проверяет порядок тегов (мягко)
        strict_format_reward_func,   # Проверяет строгое форматирование тегов с переносами строк
        path_format_reward_func,     # Проверяет, что внутри <answer> только числа и запятые
        correctness_reward_func,     # Главная логика: старт/стоп, существование ребер и оптимальность
    ],
    args=training_args,
    train_dataset=get_shortest_path_dataset('data/train_v1.pkl')
)

trainer.train()

Unsloth: We now expect `per_device_train_batch_size` * `gradient_accumulation_steps` * `world_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 161,480,704 of 7,777,097,216 (2.08% trained)


INFO 02-21 23:32:31 [peft_helper.py:58] Loading LoRA weights trained with rsLoRA.
Unsloth: Will smartly offload gradients to save VRAM!
--------------------
Target path: 2, 8, 3 | Target cost: 11
Response: 
Reward: 0.000


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / xmlcount_reward_func / mean,rewards / xmlcount_reward_func / std,rewards / soft_format_reward_func / mean,rewards / soft_format_reward_func / std,rewards / strict_format_reward_func / mean,rewards / strict_format_reward_func / std,rewards / path_format_reward_func / mean,rewards / path_format_reward_func / std,rewards / correctness_reward_func / mean,rewards / correctness_reward_func / std
1,0.000000,0.125000,0.000000,200.000000,200.000000,200.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.125000,0.000000,200.000000,200.000000,200.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


--------------------
Target path: 1, 11, 3, 5 | Target cost: 10
Response: 
Reward: 0.000
--------------------
Target path: 10, 3, 7, 11 | Target cost: 7
Response: 
Reward: 0.000


Exception ignored in: <function _xla_gc_callback at 0x7dd39607cf40>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/jax/_src/lib/__init__.py", line 127, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 
